# Generative Adversarial Networks

Generative Adversarial Networks or GANs are recent unsupervised techniques (introduced in 2014 by Ian Goodfellow) to train neural networks to generate plausible data using a zero sum game and then outputs the entire output value - such as an image in one shot.

In this lecture, I'll be building a GAN trained on the MNIST dataset

## How GANS Work:

GANs make use of two neural networks instead of one. The first neural network - called the generator - uses a differentiable function as a genenrator network. The generator takes some random noise as input and transforms it to have a recognizable structure such as a realistic image

<img src="images/2.png" alt="Drawing" style="width: 800px;"/>

### How do you train the Generator to do this?

The generator is trained differently from a normal supervised model. It's just shown a bunch of images and told to make more images that come from the same probability distribution. This is where a second neural network, called the discriminator, comes in which learns to guide the generator.

The discriminator is just a regular neural network classifier. During the training process the discriminator is shown real images from the training data half the time and fake images from the generator the other half. The discrimanor is trained to output the probability that the input is real so it tries to assign a probability near 1 to real images and a probability near 0 to fake images

<img src="images/3.png" alt="Drawing" style="width: 800px;"/>

Ian Goodfellow's orignal paper talks about the generator and discrinimator playing the following two player min max game:

<img src="images/6.png" alt="Drawing" style="width: 550px;"/>

Two optimization algorithms are run simultaneously each minimizing one of the network’s cost with respect to its parameters. The equilibrium occurs at a maximum for the discriminator and a minimum for the generator.

### How do we achieve this equilibrium?

The generator takes random values Z and maps them to output values X. Wherever the generator maps more values of Z, the probability distribution over X represented by the model becomes denser . The discriminator outputs high values wherever the density of real data is greater than the density of generated data. The generator changes the samples it produces to move uphill along the function learned by the discriminator. In other words, the generator moves its samples into areas where the model distribution is not yet dense enough.

Eventually, the generator’s distribution matches the real distribution and the discriminator has to output a probability of one half everywhere because every point is equally likely to be generated by the real dataset as to be generated by the model because the two densities are now equal as can be seen from the image above (Again, taken from Ian Goodfellow’s original paper on GANs)

<img src="images/4.png" alt="Drawing" style="width: 800px;"/>

## A GAN Model To Generate Realistic Face Images

In [1]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import os
from glob import glob
from matplotlib import pyplot
from image_loader import Imageset
import imp

### Defining the Generator and Discriminator

In [2]:
def model_inputs(image_width, image_height, image_channels, z_dim):
    
    real = tf.placeholder(tf.float32, (None, image_width, image_height, image_channels), name='real')
    z = tf.placeholder(tf.float32, shape=(None, z_dim), name='z')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return real, z, learning_rate

In [3]:
def discriminator(images, reuse=False):
    
    alpha = 0.2
    
    with tf.variable_scope('discriminator', reuse=reuse):
        
        x1 = tf.layers.conv2d(images, 128, 5, strides=2, padding='same')
        relu1 = tf.maximum(alpha * x1, x1)
        
        
        x2 = tf.layers.conv2d(relu1, 256, 5, strides=2, padding='same')
        bn2 = tf.layers.batch_normalization(x2, training=True)
        relu2 = tf.maximum(alpha * bn2, bn2)
        
        
        x3 = tf.layers.conv2d(relu2, 512, 5, strides=2, padding='same')
        bn3 = tf.layers.batch_normalization(x3, training=True)
        relu3 = tf.maximum(alpha * bn3, bn3)
        

        flat = tf.reshape(relu3, (-1, 4*4*512))
        logits = tf.layers.dense(flat, 1)
        out = tf.sigmoid(logits)
        
        return out, logits

In [4]:
def generator(z, out_channel_dim, is_train=True):
    
    alpha = 0.2
    
    with tf.variable_scope('generator', reuse = not is_train):
        
        x1 = tf.layers.dense(z, 7*7*512)
        # Reshape it to start the convolutional stack
        x1 = tf.reshape(x1, (-1, 7, 7, 512))
        x1 = tf.layers.batch_normalization(x1, training=is_train)
        x1 = tf.maximum(alpha * x1, x1)
        
        x2 = tf.layers.conv2d_transpose(x1, 256, 5, strides=2, padding='same')
        x2 = tf.layers.batch_normalization(x2, training=is_train)
        x2 = tf.maximum(alpha * x2, x2)
        
        x3 = tf.layers.conv2d_transpose(x2, 128, 5, strides=2, padding='same')
        x3 = tf.layers.batch_normalization(x3, training=is_train)
        x3 = tf.maximum(alpha * x3, x3)
        
        logits = tf.layers.conv2d_transpose(x3, out_channel_dim, 5, strides=1, 
                                            padding='same')
        
        out = tf.tanh(logits)
        return out

### Defining Losses and Optimizers

In [5]:
def model_loss(input_real, input_z, out_channel_dim):
    
    generator_model = generator(input_z, out_channel_dim)
    discriminator_model_real, discriminator_logits_real = discriminator(input_real)
    discriminator_model_fake, discriminator_logits_fake = discriminator(generator_model, reuse=True)
    
    smooth = 0.1
    
    discriminator_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=discriminator_logits_real, labels=tf.ones_like(discriminator_model_real) * (1.0 - smooth)))
    
    discriminator_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=discriminator_logits_fake, labels=tf.zeros_like(discriminator_model_fake)))
    
    generator_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=discriminator_logits_fake, labels=tf.ones_like(discriminator_model_fake)))
    
    discriminator_loss = discriminator_loss_real + discriminator_loss_fake
    
    return discriminator_loss, generator_loss

In [6]:
def model_opt(d_loss, g_loss, learning_rate, beta1):
    
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        d_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)

    return d_train_opt, g_train_opt 

### Helper method to show output during training

In [7]:
import numpy as np

def show_generator_output(sess, n_images, input_z, out_channel_dim, image_mode):
    
    cmap = None if image_mode == 'RGB' else 'gray'
    z_dim = input_z.get_shape().as_list()[-1]
    example_z = np.random.uniform(-1, 1, size=[n_images, z_dim])

    samples = sess.run(
        generator(input_z, out_channel_dim, False),
        feed_dict={input_z: example_z})

    images_grid = helper.images_square_grid(samples, image_mode)
    pyplot.imshow(images_grid, cmap=cmap)
    pyplot.show()

### Training the GAN

In [8]:
def train(epoch_count, batch_size, z_dim, learning_rate, beta1, get_batches, data_shape, data_image_mode):
    
    input_real, input_z, lr = model_inputs(data_shape[1], data_shape[2], data_shape[3], z_dim)
    discriminator_loss, generator_loss = model_loss(input_real, input_z, data_shape[3])
    d_opt, g_opt = model_opt(discriminator_loss, generator_loss, lr, beta1)

    steps = 0
    print_every = 10
    show_every = 100

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch_i in range(epoch_count):
            for batch_images in get_batches(batch_size):

                steps += 1
                batch_images = batch_images*2
                
                batch_z = np.random.uniform(-1, 1, size=(batch_size, z_dim))

                _ = sess.run(d_opt, feed_dict={input_real: batch_images, input_z: batch_z, lr: learning_rate})
                _ = sess.run(g_opt, feed_dict={input_real: batch_images, input_z: batch_z, lr: learning_rate})
            
                if steps % print_every == 0:
                    train_loss_d = discriminator_loss.eval(feed_dict={input_z: batch_z, input_real: batch_images})
                    train_loss_g = generator_loss.eval(feed_dict={input_z: batch_z})
                    print("Epoch {}..".format(epoch_i+1),
                          "Discriminator Loss: {:.4f}...".format(train_loss_d),
                          "Generator Loss: {:.4f}".format(train_loss_g))

                if steps % show_every == 0:
                    show_generator_output(sess, batch_size, input_z, data_shape[3], data_image_mode)

In [15]:
def batch_generator(data, labels, batch_size):
    rand_choice = np.random.choice(data.shape[0], batch_size, replace=False)
    
    batch_x = data[rand_choice]
    batch_y = labels[rand_choice]
    
    return batch_x, batch_y

In [10]:
batch_size = 32
z_dim = 100
learning_rate = 0.001
beta1 = 0.5
epochs = 1

# data_dir = './data'
# import helper

# celeba_dataset = helper.Dataset('celeba', glob(os.path.join(data_dir, 'img_align_celeba/*.jpg')))

# with tf.Graph().as_default():
#     train(epochs, batch_size, z_dim, learning_rate, beta1, celeba_dataset.get_batches,
#           celeba_dataset.shape, celeba_dataset.image_mode)

In [13]:
import image_loader as il
image_loader = imp.reload(il)
# MTG Card Loader testing
# Load all image directory names
images_path = "../data/images/sets"
# sets = os.listdir(images_path)

# Load preconfigured set_dimensions file
sets = pd.read_csv("../data/images/set_dimensions_big.csv",header=0)
elves_set = image_loader.Imageset(sets, 'elf', images_path)

In [ ]:
# elves_images = load_images_type_dim(sets, 'elf', images_path)

In [14]:
elves_images = elves_set.get_images()
len(elves_images)

435

In [ ]:
batch_a = celeba_dataset.get_batches(16)

for image in batch_a:
    print(image.shape)
    print('a')
    break

### GAN Resources

Before I show some applications of GANs, here are some resources to get started with GANS:

* The original [paper](https://arxiv.org/pdf/1406.2661.pdf) written by Ian Goodfellow in 2014. 
* Siraj Raval's [video tutorial](https://www.youtube.com/watch?v=-E2N1kQc8MM) on GANs (Really fun video)
* This other [video](https://www.youtube.com/watch?v=deyOX6Mt_As) on GANS by Siraj Raval (Another fun video)
* Ian Godfellow's [keynote](https://channel9.msdn.com/Events/Neural-Information-Processing-Systems-Conference/Neural-Information-Processing-Systems-Conference-NIPS-2016/Generative-Adversarial-Networks)

### Applications of GANS

Cool Applications of GANS:
* [Generative Adversarial Text to Image Synthesis](https://github.com/paarthneekhara/text-to-image)
* [Image super-resolution through deep learning](https://github.com/david-gpu/srez)
* [Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks (CycleGAN)](https://github.com/junyanz/CycleGAN)
* [Image-to-image translation with conditional adversarial nets (pix2pix)](https://github.com/phillipi/pix2pix)
* [Deep Convolutional Generative Adversarial Networks (DCGAN)](https://github.com/Newmu/dcgan_code)
* [Τensorflow implementation of Deep Convolutional Generative Adversarial Networks (DCGAN)](https://github.com/carpedm20/DCGAN-tensorflow)
* [Generative Visual Manipulation on the Natural Image Manifold (iGAN)](https://github.com/junyanz/iGAN)
* [Neural Photo Editing with Introspective Adversarial Networks](https://github.com/ajbrock/Neural-Photo-Editor)